# Data Load & Simple Preprocessing

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
path = '/content/gdrive/MyDrive/dacon/jobcare/'
import os
os.listdir(path)

['X_train.csv',
 'X_test.csv',
 'prediction_12242.csv',
 '참가자_제공_레이아웃.pdf',
 '속성_H_코드.csv',
 '속성_D_코드.csv',
 '속성_L_코드.csv',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'X_train_mean.csv',
 'X_test_mean.csv',
 'y_train.csv']

In [7]:
import pandas as pd

In [8]:
train = pd.read_csv(path + "train.csv")
X_test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

d_code = pd.read_csv(path + '속성_D_코드.csv', index_col=0).T.to_dict()
h_code = pd.read_csv(path + '속성_H_코드.csv', index_col=0).T.to_dict()
l_code = pd.read_csv(path + '속성_L_코드.csv', index_col=0).T.to_dict()

분류코드에 따라 아래와 같이 구분자를 붙혀 구분합니다.
- n: 세분류코드 
- s: 소분류코드   
- m: 중분류코드 
- l: 대분류코드 
- u: 상위코드

In [9]:
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

train = add_code(train, d_code, h_code, l_code)
X_test = add_code(X_test, d_code, h_code, l_code)
print("train_data.shape: ", train.shape)
print("test_data.shape: ", X_test.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 63 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      501951 non-null  int64 
 1   d_l_match_yn            501951 non-null  bool  
 2   d_m_match_yn            501951 non-null  bool  
 3   d_s_match_yn            501951 non-null  bool  
 4   h_l_match_yn            501951 non-null  bool  
 5   h_m_match_yn            501951 non-null  bool  
 6   h_s_match_yn            501951 non-null  bool  
 7   person_attribute_a      501951 non-null  int64 
 8   person_attribute_a_1    501951 non-null  int64 
 9   person_attribute_b      501951 non-null  int64 
 10  person_prefer_c         501951 non-null  int64 
 11  person_prefer_d_1       501951 non-null  int64 
 12  person_prefer_d_2       501951 non-null  int64 
 13  person_prefer_d_3       501951 non-null  int64 
 14  person_prefer_e         501951 non-n

In [11]:
data = [train, X_test]

for i in range(2):
  data[i].contents_open_dt = pd.to_datetime(data[i].contents_open_dt)

  data[i]['month'] = data[i].contents_open_dt.dt.month
  data[i]['day'] = data[i].contents_open_dt.dt.day
  data[i]['week'] = data[i].contents_open_dt.dt.isocalendar().week
  data[i]['dayofweek'] = data[i].contents_open_dt.dt.dayofweek
  data[i]['hour'] = data[i].contents_open_dt.dt.hour
  data[i]['minute'] = data[i].contents_open_dt.dt.minute

  data[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

In [12]:
for i in range(train.shape[1]):
  train.iloc[:,i] = train.iloc[:,i].astype(int)
for i in range(X_test.shape[1]):
  X_test.iloc[:,i] = X_test.iloc[:,i].astype(int)

In [13]:
X_train = train.drop('target',axis=1)
y_train = train['target']

In [14]:
X_train = X_train.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)
X_test = X_test.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)


# 변수처리

In [11]:
X_train["kfold"] = -1

In [12]:
from sklearn import model_selection
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=X_train)):
    X_train.loc[valid_indicies, "kfold"] = fold

In [13]:
object_cols = X_train.columns[:-1]
df = pd.concat([X_train, y_train], axis=1)

for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain =  df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)
        
        # 명목형 변수를 그냥 라벨 처리하기에는 한계가 있으므로 각 값 별로 타겟값의 평균을 대입해주고자 한다.
        # 이때 과적합을 막기위해 5-foldCV를 활용하여 각 폴드별 값의 평균을 대입해준다.
        feat = xtrain.groupby(col)["target"].agg("mean")
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = X_test[col].map(feat)
        else:
            temp_test_feat += X_test[col].map(feat)
    
    temp_test_feat /= 5
    X_test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df)
    

X_train = df.copy()

In [14]:
X_train = X_train.drop(['kfold', 'target'],axis=1)

In [15]:
for i in range(X_train.shape[1]):
    X_train.iloc[:,i] = X_train.iloc[:,i].fillna(X_train.iloc[:,i].mean())
    X_test.iloc[:,i] = X_test.iloc[:,i].fillna(X_train.iloc[:,i].mean())

In [20]:
X_train_mean = X_train.iloc[:,62:]
X_test_mean = X_test.iloc[:,62:]

# LGBM

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tqdm import tnrange, tqdm_notebook, notebook, tqdm
import time
import random

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, KFold

from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import f1_score

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [57]:
model = LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier()

In [68]:
pd.Series(model.feature_importances_,X_train.columns).sort_values(ascending=True).iloc[:17]

contents_attribute_h_l    0
person_prefer_h_3_l       0
contents_attribute_j      0
person_prefer_d_1_s       0
person_prefer_d_1_m       0
person_prefer_d_1_l       0
person_prefer_d_2_s       0
person_prefer_d_2_m       0
person_prefer_d_2_l       0
person_prefer_d_3_s       0
person_prefer_d_3_m       0
contents_attribute_d_s    0
contents_attribute_d_m    0
contents_attribute_d_l    0
contents_attribute_h_m    0
person_prefer_h_2_l       0
person_prefer_d_3_l       0
dtype: int64

In [69]:
pd.Series(model.feature_importances_,X_train.columns).sort_values(ascending=False).iloc[:20]

contents_attribute_h            86
tar_enc_minute                  82
tar_enc_contents_attribute_k    79
minute                          78
tar_enc_contents_attribute_d    77
tar_enc_week                    68
tar_enc_day                     67
contents_attribute_l            65
tar_enc_person_prefer_d_2       64
tar_enc_contents_attribute_h    63
tar_enc_contents_attribute_l    62
hour                            56
tar_enc_contents_attribute_c    55
tar_enc_person_prefer_h_2       55
tar_enc_person_attribute_a      54
tar_enc_person_prefer_d_3       53
person_prefer_d_2               51
tar_enc_person_prefer_h_1       49
contents_attribute_d            48
tar_enc_hour                    48
dtype: int64

In [81]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [3]:
from catboost import Pool,CatBoostClassifier

In [83]:
X_train = X_train.iloc[:,0:62]
cat_features = X_train.columns[X_train.nunique() > 2].tolist()

is_holdout = False
n_splits = 5
iterations = 500
patience = 50

model = CatBoostClassifier(iterations=iterations,random_state=123,
                           cat_features=cat_features, one_hot_max_size=4)
model.fit(X_train, y_train)

Learning rate set to 0.276797
0:	learn: 0.6931365	total: 9.49s	remaining: 1h 18m 55s
1:	learn: 0.6931151	total: 13.9s	remaining: 57m 42s
2:	learn: 0.6930890	total: 19.2s	remaining: 53m 6s
3:	learn: 0.6930638	total: 23.9s	remaining: 49m 26s
4:	learn: 0.6930230	total: 28.7s	remaining: 47m 16s
5:	learn: 0.6930144	total: 33.1s	remaining: 45m 23s
6:	learn: 0.6929813	total: 37.9s	remaining: 44m 30s
7:	learn: 0.6929695	total: 42.5s	remaining: 43m 36s
8:	learn: 0.6929445	total: 47.8s	remaining: 43m 28s
9:	learn: 0.6929296	total: 52s	remaining: 42m 30s
10:	learn: 0.6929109	total: 56.7s	remaining: 42m
11:	learn: 0.6928861	total: 1m 1s	remaining: 41m 27s
12:	learn: 0.6928694	total: 1m 5s	remaining: 40m 48s
13:	learn: 0.6928365	total: 1m 9s	remaining: 40m 23s
14:	learn: 0.6928172	total: 1m 14s	remaining: 39m 58s
15:	learn: 0.6928050	total: 1m 18s	remaining: 39m 34s
16:	learn: 0.6927881	total: 1m 23s	remaining: 39m 36s
17:	learn: 0.6927881	total: 1m 24s	remaining: 37m 46s
18:	learn: 0.6927777	total

In [85]:
pd.Series(model.feature_importances_,X_train.columns).sort_values(ascending=True).iloc[:20]

contents_attribute_k      0.010198
h_s_match_yn              0.022790
d_m_match_yn              0.025261
d_s_match_yn              0.107210
contents_attribute_c      0.110214
contents_attribute_j      0.166002
h_m_match_yn              0.218734
h_l_match_yn              0.223196
person_attribute_a        0.265188
d_l_match_yn              0.420965
contents_attribute_a      0.559144
contents_attribute_i      0.598577
month                     1.007271
person_prefer_d_3_m       1.228405
contents_attribute_d_l    1.255913
person_prefer_h_2_m       1.310975
contents_attribute_h_m    1.360736
person_prefer_d_2_m       1.365943
person_attribute_a_1      1.387063
contents_attribute_h_l    1.425039
dtype: float64

In [86]:
pd.Series(model.feature_importances_,X_train.columns).sort_values(ascending=False).iloc[:20]

contents_attribute_l_s    3.037576
week                      2.792591
hour                      2.726993
contents_attribute_j_1    2.716711
person_prefer_d_3         2.560148
person_prefer_d_2         2.540283
contents_attribute_l      2.499006
person_prefer_h_3_m       2.464735
contents_attribute_l_m    2.427376
contents_attribute_d_n    2.419998
person_prefer_h_2         2.361386
contents_attribute_h      2.289746
person_prefer_e           2.251856
person_prefer_d_1         2.237035
person_prefer_d_2_n       2.192296
person_prefer_d_1_n       2.185965
dayofweek                 2.176124
contents_attribute_e      2.139223
person_prefer_h_2_l       2.136211
contents_attribute_d_s    2.091836
dtype: float64

In [87]:
X_train_mean.to_csv(path + 'X_train_mean.csv')
X_test_mean.to_csv(path + 'X_test_mean.csv')
y_train.to_csv(path + 'y_train.csv')

In [88]:
X_train_mean = pd.read_csv(path + "X_train_mean.csv")
X_test_mean = pd.read_csv(path + "X_test_mean.csv")
y_train = pd.read_csv(path + "y_train.csv")

In [90]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=d492d2bec08fe1a93ab9db7d48a036b33d72dff0d972eeb581f33c064f950aa0
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [91]:
from bayes_opt import BayesianOptimization

In [92]:
# create purpose function
def lgbm_cv(learning_rate, n_estimators, max_depth, num_leaves, subsample, min_child_weight, colsample_bytree, max_bin, reg_alpha, reg_lambda):
    model = LGBMClassifier(learning_rate = learning_rate,
                                n_estimators = int(n_estimators),
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth)),
                                n_jobs = -1,
                                random_state = 0,
                                subsample = max(subsample, 0),
                                min_child_weight = int(round(min_child_weight)),
                                colsample_bytree = colsample_bytree,
                                max_bin = int(round(max_bin)),
                                reg_alpha = max(reg_alpha, 0),
                                reg_lambda = max(reg_lambda, 0)
                               )
    scores = cross_validate(model, X_train_mean, y_train, cv=5, n_jobs = -1, scoring='f1')
    return np.mean(scores['test_score'])

In [98]:
# Interval to be explored for input values
pbounds = {'learning_rate': (0.005, 0.1),
           'n_estimators': (100, 2000),
           'max_depth': (5, 50),
           'num_leaves': (0, 100),
           'subsample': (0, 0.99),
           'min_child_weight' : (0, 100),
           'colsample_bytree': (0, 0.99),
           'max_bin': (0, 1000),
           'reg_alpha': (0, 1),
           'reg_lambda' : (0, 1)
          }

In [99]:
lgbmBO = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 0)

In [100]:
lgbmBO.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

|   iter    |  target   | colsam... | learni... |  max_bin  | max_depth | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.3006   |  0.5433   |  0.07294  |  602.8    |  29.52    |  42.37    |  1.327e+0 |  43.76    |  0.8918   |  0.9637   |  0.3796   |
|  2        |  0.2781   |  0.7838   |  0.05525  |  568.0    |  46.65    |  7.104    |  265.5    |  2.022    |  0.8326   |  0.7782   |  0.8613   |
|  3        |  0.2919   |  0.9688   |  0.08092  |  461.5    |  40.12    |  11.83    |  1.316e+0 |  14.34    |  0.9447   |  0.5218   |  0.4105   |
|  4        |  0.2989   |  0.2619   |  0.07855  |  456.2    |  30.58    |  1.879    |  1.274e+0 |  61.21    |  0.6169   |  0.9437   |  0.675    |
|  5        |  0.2901   |  0.3559   |  0.04652  |  697.6    |  7.71     |  66.68    |  1.374e+0 |  21.04    |  0.1289   |  0

KeyboardInterrupt: ignored

In [15]:
f1 =[]
model_names =[]
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

model = CatBoostClassifier(verbose=False,random_state=0)

model.fit(X_tra, y_tra, cat_features=categorical_features_indices, eval_set=(X_val, y_val))
y_pred = model.predict(X_val)
f1_score.append(round(accuracy_score(y_val, y_pred),4))

model_names = ['Catboost_default']
result = pd.DataFrame({'f1':f1_score}, index=model_names)
result

AttributeError: ignored

In [ ]:
def objective(trial):

    categorical_features_indices = np.where(X.dtypes != np.float)[0]
    
    X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = CatBoostClassifier(**param)

    cat_cls.fit(X_tra, y_tra, eval_set=[(X_val, y_val)], cat_features=categorical_features_indices,verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_val)
    pred_labels = np.rint(preds)
    f1 = f1_score(y_val, pred_labels)
    return f1


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
f1 =[]
model_names =[]

categorical_features_indices = np.where(X.dtypes != np.float)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = CatBoostClassifier(verbose=False,random_state=0,
                          objective= 'CrossEntropy',
    colsample_bylevel= 0.04292240490294766,
    depth= 10,
    boosting_type= 'Plain',
    bootstrap_type= 'MVS')

model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test))
y_pred = model.predict(X_test)
accuracy.append(round(accuracy_score(y_test, y_pred),4))
print(classification_report(y_test, y_pred))

model_names = ['Catboost_tuned']
result_df6 = pd.DataFrame({'Accuracy':accuracy}, index=model_names)
result_df6